In [1]:
import black
import fasttext
import jupyter_black
import json
import keras
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import seaborn as sns
import tensorflow as tf
import random
import re
from collections import Counter
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input, CuDNNLSTM, Dense
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Bidirectional,
    Dense,
    Dropout,
    Embedding,
    LSTM,
    GRU
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    cross_val_score,
    train_test_split,
    StratifiedKFold,
)
import xml.etree.ElementTree as ET

# Setting options
pd.options.mode.chained_assignment = None  # default='warn'
nltk.download("stopwords")
nltk.download("punkt")
#black for linting reasons
jupyter_black.load(
    lab=False,
    line_length=80,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)
pd.set_option('display.max_colwidth', None)
print("All good!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
DEBUG:jupyter_black:config: {'line_length': 80, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

All good!


In [2]:
csv_file_path = r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\DataSets\open-dataset-for-sentiment-analysis-master\betsentiment-ES-tweets-sentiment-worldcup.csv"


# Function to extract sentiment scores from the json str
def extract_sentiment_scores(json_str):
    sentiment_data = json.loads(json_str)
    return (
        sentiment_data["Neutral"],
        sentiment_data["Negative"],
        sentiment_data["Positive"],
        sentiment_data["Mixed"],
    )


# Define data types
dtype_dict = {
    "tweet_date_created": str,
    "tweet_id": int,
    "tweet_text": str,
    "language": str,
    "sentiment": str,
}

try:
    df = pd.read_csv(csv_file_path, encoding="utf-8", dtype=dtype_dict)
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_path, encoding="latin-1", dtype=dtype_dict)

sentiment_scores_list = df["sentiment_score"].map(extract_sentiment_scores)

sentiment_scores_df = pd.DataFrame(
    sentiment_scores_list.tolist(),
    columns=["Neutral", "Negative", "Positive", "Mixed"],
)

df = pd.concat([df, sentiment_scores_df], axis=1)

df.drop("sentiment_score", axis=1, inplace=True)

In [ ]:
df

In [3]:
# Read the stopwords from the .txt file and store them in a list
custom_stopwords_file = (
    r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\utilities\stopwords.txt"
)
with open(custom_stopwords_file, "r", encoding="utf-8") as file:
    custom_stopwords_list = [line.strip() for line in file]

# Create an empty set to hold the stopwords
custom_stopwords_set = set()

# Add the stopwords from the list to the set
custom_stopwords_set.update(custom_stopwords_list)

In [4]:
# Specify the Snowball stemmer for Spanish
stemmer = SnowballStemmer("spanish")

# 01 - Convert NaN values to an empty string
df["tweet_text"] = df["tweet_text"].fillna("")

# 02 - Lowercasing
df["tweet_text"] = df["tweet_text"].str.lower()

# 03 - Remove links (URLs)
df["tweet_text"] = df["tweet_text"].apply(
    lambda text: re.sub(r"http\S+|www\S+|https\S+", "", text)
)

# 04 - Removing punctuation
df["tweet_text"] = df["tweet_text"].str.replace("[^\w\s]", "")

# 05 - Tokenization
df["tokens"] = df["tweet_text"].apply(word_tokenize)

# 06 - Removing stop words
stop_words = set(stopwords.words("spanish"))
df["filtered_tokens"] = df["tokens"].apply(
    lambda tokens: [word for word in tokens if word not in stop_words]
)

# 07 - Stemming
df["stemmed_tokens"] = df["filtered_tokens"].apply(
    lambda tokens: [stemmer.stem(word) for word in tokens]
)

C:\Users\Cacu\AppData\Local\Temp\ipykernel_12688\3792906956.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df["tweet_text"] = df["tweet_text"].str.replace("[^\w\s]", "")


In [6]:
original_df = df

## 1. Embedding

fastText es una biblioteca para el aprendizaje de incrustaciones de palabras y clasificación de texto creada por el laboratorio de Investigación de Inteligencia Artificial de Facebook (FAIR). El modelo es un algoritmo de aprendizaje no supervisado para obtener representaciones vectoriales de palabras. Facebook pone a disposición modelos preentrenados para 294 idiomas. fastText utiliza una red neuronal para la incrustación de palabras [Fuente: Wikipedia].

Documentación sobre Gensim: models.fasttext

FastText es una extensión de Word2Vec propuesta por Facebook en 2016. En lugar de alimentar palabras individuales en la Red Neuronal, FastText divide las palabras en varios n-gramas (subpalabras). Por ejemplo, los trigramas para la palabra "manzana" son "man," "anz," y "nza" (ignorando los límites de inicio y fin de las palabras). El vector de incrustación de la palabra "manzana" será la suma de todos estos n-gramas. Después de entrenar la Red Neuronal, tendremos incrustaciones de palabras para todos los n-gramas dados el conjunto de datos de entrenamiento. Las palabras raras ahora pueden representarse adecuadamente, ya que es muy probable que algunos de sus n-gramas también aparezcan en otras palabras. Te mostraré cómo usar FastText con Gensim en la siguiente sección.

In [ ]:
df["tokens_as_string"] = df["tokens"].apply(" ".join)
# Save the stemmed tokens to a text file (one sentence per line)
with open("tokens.txt", "w") as f:
    f.write("\n".join(df["tokens_as_string"]))

In [ ]:
# Fasttext model on stemmed tokens
model = fasttext.train_unsupervised("tokens.txt", model="skipgram")

# Create tweet-level embeddings using the trained model
embeddings = []

In [ ]:
for tokens in df["tokens_as_string"]:
    vector = model.get_sentence_vector(tokens)
    embeddings.append(vector)

# Convert the embeddings to a DataFrame
embedding_df = pd.DataFrame(embeddings)

In [ ]:
# embeddings df with the original dataset
df_worldcup_embeddings = pd.concat([df, embedding_df], axis=1)

# embedding df to csv
df_worldcup_embeddings.to_csv("2018_dataset_with_embeddings.csv", index=False)

model.save_model("model_worldcup_embedding.bin")

In [ ]:
df_worldcup_embeddings

In [ ]:
nrows_to_load = 1000000  # Adjust this to the desired subset size

# start from here - csv already generated
df = pd.read_csv("2018_dataset_with_embeddings.csv", nrows=nrows_to_load)
# shuffle the DataFrame rows
# df = df.sample(frac=1)

## Preparing for training

In [ ]:
df.count()

In [5]:
label_mapping = {"POSITIVE": 2, "NEUTRAL": 1, "NEGATIVE": 0}
df["sentiment_encoded"] = df["sentiment"].map(label_mapping)

In [6]:
# Create separate DataFrames for each class
positive_df = df[df["sentiment_encoded"] == 2]
neutral_df = df[df["sentiment_encoded"] == 1]
negative_df = df[df["sentiment_encoded"] == 0]

In [7]:
print(df.isnull().sum())

# Remove rows with missing values
df.dropna(inplace=True)

# Remove duplicates
df.drop_duplicates(subset=["tweet_text"], keep="first", inplace=True)

tweet_date_created       0
tweet_id                 0
tweet_text               0
language                 0
sentiment                0
Neutral                  0
Negative                 0
Positive                 0
Mixed                    0
tokens                   0
filtered_tokens          0
stemmed_tokens           0
sentiment_encoded     7606
dtype: int64


In [8]:
# Sample 25k rows from each class
num_samples = 25000
positive_sample = positive_df.sample(n=num_samples, random_state=42)
neutral_sample = neutral_df.sample(n=num_samples, random_state=42)
negative_sample = negative_df.sample(n=num_samples, random_state=42)

In [9]:
# Concatenate the samples to create a balanced subset
subset_df = pd.concat([positive_sample, neutral_sample, negative_sample])

# Shuffle the subset to randomize the order
subset_df = subset_df.sample(frac=1, random_state=42)

df = subset_df

In [ ]:
# Extract X (features) and y (target)
X = df.iloc[:, 13:-1].to_numpy()
y = df["sentiment_encoded"].to_numpy()

# Make sure y is in the correct shape
y = np.reshape(y, (-1, 1))

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
# Initialize a Tokenizer
tokenizer = Tokenizer()
# Fit the tokenizer on your tokenized text data
tokenizer.fit_on_texts(df["tokens"])

# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the zero padding

# Print the vocabulary size
print("Vocabulary size (input_dim):", vocab_size)

In [ ]:
# Bidirectional LSTM

# Define the model
model2 = Sequential()
model2.add(
    Embedding(
        input_dim=vocab_size,
        output_dim=100,
        input_length=100,
    )
)
model2.add(Bidirectional(LSTM(128, return_sequences=True)))
model2.add(Bidirectional(LSTM(100)))
model2.add(Dense(128,activation='tanh'))
model2.add(
    Dropout(0.2)
)
model2.add(Dense(32,activation='relu'))
model2.add(Dense(3, activation="softmax"))

In [ ]:
# Compile the model
model2.compile(
    loss=SparseCategoricalCrossentropy(),
    optimizer=Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

# Train the model
history = model2.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=256,
    validation_split=0.2,
#    callbacks=[early_stopping],
)

# Evaluate the model
loss, accuracy = model2.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

In [ ]:
# TODO: tirar los word embedding post traerse el dataset

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'cv'], loc='upper left')
plt.show()

In [ ]:
prob = model.predict(X_test)
y_pred = []
for i in range(prob.shape[0]):
  if(prob[i][0]>=0.5):
    y_pred.append(1)
  else:
    y_pred.append(0)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
# other word embedding - since fasttext unsupervised model not working as expected
import fasttext.util

# fasttext.util.download_model('es')

In [ ]:
# FastText word embeddings model
model = fasttext.load_model("cc.es.300.bin")

In [7]:
subset_size = 150000
random_subset = df.sample(
    n=subset_size, random_state=42
)  # Adjust the random_state as needed

df = random_subset

In [ ]:
df

In [ ]:
# Create an empty NumPy array
empty_array = np.empty((len(df),), dtype=object)

# Assign the empty array to a new column in your DataFrame
df["vector"] = empty_array

In [ ]:
for row in df.index:
    # Calculate the embedding for each token in the tweet
    tweet_vector = np.mean(
        [model.get_word_vector(token) for token in df["tokens"][row]], axis=0
    )

    # Store the tweet embeddings in the dataframe
    df["vector"][row] = tweet_vector

In [ ]:
df

In [ ]:
model.get_word_vector("Messi")

In [9]:
label_mapping = {"POSITIVE": 2, "NEUTRAL": 1, "NEGATIVE": 0}
df["sentiment_encoded"] = df["sentiment"].map(label_mapping)

In [ ]:
# optional: separate DataFrames for each class
positive_df = df[df['sentiment_encoded'] == 2]
neutral_df = df[df['sentiment_encoded'] == 1]
negative_df = df[df['sentiment_encoded'] == 0]

In [ ]:
# Sample 25k rows from each class
num_samples = 25000
positive_sample = positive_df.sample(n=num_samples, random_state=42)
neutral_sample = neutral_df.sample(n=num_samples, random_state=42)
negative_sample = negative_df.sample(n=num_samples, random_state=42)

In [ ]:
print(df.isnull().sum())

# Remove rows with missing values
df.dropna(inplace=True)

# Remove duplicates
df.drop_duplicates(subset=["tweet_text"], keep="first", inplace=True)

In [10]:
import gensim, logging, os
from gensim.models.word2vec import Word2Vec

model = gensim.models.Word2Vec(
    df["tokens"], min_count=10, vector_size=100, workers=2
)

In [11]:
model.save("MODEL_WORD2VEC.w2v")

In [12]:
print(model.corpus_count)

75000


In [13]:
# Create a vocabulary based on the vectors
vocab = {word: index for index, word in enumerate(model.wv.index_to_key)}

# Define the index for the 'unknown' token
unknown_token_index = len(vocab)  # Assuming it's the next index


# Function to map words to indices, with handling for out-of-vocabulary words
def words_to_indices(vec):
    return [vocab.get(word, unknown_token_index) for word in vec]


# Apply the mapping function to your 'tokens' column
df["sequence"] = df["tokens"].apply(words_to_indices)

In [14]:
df

,tweet_date_created,tweet_id,tweet_text,language,sentiment,Neutral,Negative,Positive,Mixed,tokens,filtered_tokens,stemmed_tokens,sentiment_encoded,sequence
107374,2018-07-02T12:40:07.432000,-740831232,buenos días a todos \noye miseleccionmx acá creímos y sí pudimos también creemos que van a ganar ahora les toca a ustedes creer y vencer \njuegaméxico,es,NEUTRAL,0.640794,0.012900,0.321520,0.024786,"[buenos, días, a, todos, oye, miseleccionmx, acá, creímos, y, sí, pudimos, también, creemos, que, van, a, ganar, ahora, les, toca, a, ustedes, creer, y, vencer, juegaméxico]","[buenos, días, oye, miseleccionmx, acá, creímos, pudimos, creemos, van, ganar, ahora, toca, ustedes, creer, vencer, juegaméxico]","[buen, dias, oye, miseleccionmx, aca, creim, pud, cre, van, gan, ahor, toc, usted, cre, venc, juegamex]",1.0,"[244, 181, 2, 41, 1232, 8, 445, 9315, 4, 198, 4451, 152, 1395, 1, 110, 2, 86, 78, 74, 546, 2, 84, 556, 4, 2176, 1084]"
826058,2018-05-08T20:01:17,492277760,record_mexico miseleccionmx tiene toda la razon el piojo miguelherreradt es un momento vital para ver el nivel de los mejores en el futbol mexicano a 38 días del mundial,es,POSITIVE,0.466427,0.020149,0.495606,0.017818,"[record_mexico, miseleccionmx, tiene, toda, la, razon, el, piojo, miguelherreradt, es, un, momento, vital, para, ver, el, nivel, de, los, mejores, en, el, futbol, mexicano, a, 38, días, del, mundial]","[record_mexico, miseleccionmx, toda, razon, piojo, miguelherreradt, momento, vital, ver, nivel, mejores, futbol, mexicano, 38, días, mundial]","[record_mex, miseleccionmx, tod, razon, pioj, miguelherreradt, moment, vital, ver, nivel, mejor, futbol, mexican, 38, dias, mundial]",2.0,"[816, 8, 62, 117, 3, 3213, 5, 3475, 3286, 9, 15, 186, 9315, 16, 60, 5, 284, 0, 10, 192, 7, 5, 230, 325, 2, 5205, 181, 20, 30]"
98555,2018-06-12T03:18:53,823517185,miseleccionmx marcofabian_10 que bien ya las escortos saben a donde ir gran detalle,es,POSITIVE,0.114220,0.021354,0.810466,0.053959,"[miseleccionmx, marcofabian_10, que, bien, ya, las, escortos, saben, a, donde, ir, gran, detalle]","[miseleccionmx, marcofabian_10, bien, escortos, saben, ir, gran, detalle]","[miseleccionmx, marcofabian_10, bien, escort, sab, ir, gran, detall]",2.0,"[8, 477, 1, 52, 27, 33, 9315, 391, 2, 201, 177, 91, 2410]"
240234,2018-06-26T16:03:45.065000,1293389825,argentina eso espero los amigos de messi no vayan a llorar si quedan afuera póngan lo que no tienen,es,NEGATIVE,0.423679,0.431658,0.061877,0.082786,"[argentina, eso, espero, los, amigos, de, messi, no, vayan, a, llorar, si, quedan, afuera, póngan, lo, que, no, tienen]","[argentina, espero, amigos, messi, vayan, llorar, si, quedan, afuera, póngan]","[argentin, esper, amig, messi, vay, llor, si, qued, afuer, pong]",0.0,"[17, 66, 204, 10, 450, 0, 100, 6, 547, 2, 626, 21, 1028, 677, 9315, 14, 1, 6, 124]"
104961,2018-07-02T03:43:36.696000,-694767616,que miseleccionmx no me parta el corazón mañana que ando malita después de hoy,es,NEGATIVE,0.279599,0.559524,0.074039,0.086838,"[que, miseleccionmx, no, me, parta, el, corazón, mañana, que, ando, malita, después, de, hoy]","[miseleccionmx, parta, corazón, mañana, ando, malita, después, hoy]","[miseleccionmx, part, corazon, mañan, ando, malit, despues, hoy]",0.0,"[1, 8, 6, 22, 9315, 5, 213, 81, 1, 2805, 9315, 200, 0, 35]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381994,2018-06-22T19:13:32,123179014,rusia2018 la derrota argentina reflejada en las portadas de los diarios de todo el mundo,es,NEUTRAL,0.943339,0.009795,0.042435,0.004431,"[rusia2018, la, derrota, argentina, reflejada, en, las, portadas, de, los, diarios, de, todo, el, mundo]","[rusia2018, derrota, argentina, reflejada, portadas, diarios, mundo]","[rusia2018, derrot, argentin, reflej, port, diari, mund]",1.0,"[31, 3, 959, 17, 9315, 7, 33, 9315, 0, 10, 7869, 0, 29, 5, 92]"
13063,2018-05-28T22:46:29,-1118453760,miseleccionmx cocacolamx con sarahpernight porque es la quien me cae bien jaja diacocacola miseleccionmx cocacolamx \nf

In [15]:
# Extract X (features) and y (target)
X = df["sequence"]
y = df["sentiment_encoded"].to_numpy()

In [16]:
y

array([1., 2., 2., ..., 0., 2., 2.])

In [17]:
# y in the correct shape
y = np.reshape(y, (-1, 1))

In [18]:
vocabulary = set()

for tokens in df["tokens"]:
    vocabulary.update(tokens)

In [19]:
vocab_size = 143778
print("Vocabulary size:", vocab_size)

Vocabulary size: 143778


In [20]:
max_sequence_length = 40
X = pad_sequences(X, maxlen=max_sequence_length)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [22]:
# Bidirectional LSTM

# Define the model
model2 = Sequential()
model2.add(
    Embedding(
        input_dim=vocab_size,
        output_dim=100,
        input_length=40,
    )
)
model2.add(Bidirectional(LSTM(64, return_sequences=True)))
model2.add(Bidirectional(LSTM(64, return_sequences=True)))
model2.add(Bidirectional(LSTM(100)))
model2.add(Dense(128, activation="tanh"))
model2.add(Dropout(0.2))
model2.add(Dense(32, activation="relu"))
model2.add(Dense(3, activation="softmax"))

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Compile the model
model2.compile(
    loss=SparseCategoricalCrossentropy(),
    optimizer=Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

# Train the model
history = model2.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=256,
    validation_split=0.2,
    #    callbacks=[early_stopping],
)

# Evaluate the model
loss, accuracy = model2.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Epoch 1/30
188/188 [==============================] - 86s 425ms/step - loss: 0.6926 - accuracy: 0.6882 - val_loss: 0.5439 - val_accuracy: 0.7744
Epoch 2/30
188/188 [==============================] - 78s 414ms/step - loss: 0.4502 - accuracy: 0.8240 - val_loss: 0.5310 - val_accuracy: 0.7899
Epoch 3/30
188/188 [==============================] - 78s 414ms/step - loss: 0.3818 - accuracy: 0.8533 - val_loss: 0.5711 - val_accuracy: 0.7825
Epoch 4/30
188/188 [==============================] - 78s 416ms/step - loss: 0.3328 - accuracy: 0.8738 - val_loss: 0.6061 - val_accuracy: 0.7765
Epoch 5/30
188/188 [==============================] - 78s 418ms/step - loss: 0.2867 - accuracy: 0.8939 - val_loss: 0.6291 - val_accuracy: 0.7698
Epoch 6/30
188/188 [==============================] - 78s 416ms/step - loss: 0.2408 - accuracy: 0.9137 - val_loss: 0.7040 - val_accuracy: 0.7679
Epoch 7/30
188/188 [==============================] - 78s 415ms/step - loss: 0.1999 - accuracy: 0.9292 - val_loss: 0.7810 - val_ac